In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']

In [3]:
class HeavierSimpleCNN(nn.Module):
    def __init__(self):
        super(HeavierSimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(x.size(0), -1)  # Ensure that the batch size is maintained
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [4]:
# Transforms and DataLoader setup
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [5]:
# Dataset and DataLoader
train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

In [6]:
train_loader = DataLoader(train_data, batch_size=20, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=20, shuffle=False, drop_last=True)

In [7]:
# Initialize the model, loss criterion and optimizer
model = HeavierSimpleCNN()
if torch.cuda.is_available():
    model.cuda()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {running_loss / len(train_loader.dataset):.4f}')


In [10]:
# Function to evaluate the model
def evaluate_model(model, val_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

In [11]:
# Train and evaluate the model
train_model(
    model,
    train_loader,
    criterion,
    optimizer,
    100 # number of epochs
)

Epoch 1/100 - Loss: 1.2822
Epoch 2/100 - Loss: 0.7601
Epoch 3/100 - Loss: 0.6604
Epoch 4/100 - Loss: 0.6726
Epoch 5/100 - Loss: 0.6764
Epoch 6/100 - Loss: 0.6666
Epoch 7/100 - Loss: 0.6303
Epoch 8/100 - Loss: 0.6465
Epoch 9/100 - Loss: 0.6535
Epoch 10/100 - Loss: 0.6523
Epoch 11/100 - Loss: 0.6408
Epoch 12/100 - Loss: 0.6412
Epoch 13/100 - Loss: 0.6360
Epoch 14/100 - Loss: 0.6425
Epoch 15/100 - Loss: 0.6192
Epoch 16/100 - Loss: 0.6518
Epoch 17/100 - Loss: 0.6535
Epoch 18/100 - Loss: 0.6405
Epoch 19/100 - Loss: 0.6544
Epoch 20/100 - Loss: 0.6447
Epoch 21/100 - Loss: 0.6393
Epoch 22/100 - Loss: 0.6465
Epoch 23/100 - Loss: 0.6412
Epoch 24/100 - Loss: 0.6505
Epoch 25/100 - Loss: 0.6264
Epoch 26/100 - Loss: 0.6216
Epoch 27/100 - Loss: 0.6210
Epoch 28/100 - Loss: 0.6749
Epoch 29/100 - Loss: 0.6783
Epoch 30/100 - Loss: 0.6436
Epoch 31/100 - Loss: 0.6285
Epoch 32/100 - Loss: 0.6272
Epoch 33/100 - Loss: 0.6305
Epoch 34/100 - Loss: 0.6315
Epoch 35/100 - Loss: 0.6472
Epoch 36/100 - Loss: 0.6251
E

In [12]:
evaluate_model(model, val_loader)

Validation Accuracy: 60.00%
